### **API 키 및 기타 설정**

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
# os.environ['ANTHROPIC_API_KEY'] = "YOUR_API_KEY"


### **필수 라이브러리 불러오기**

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai_tools import SerperDevTool

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

from langchain_openai import ChatOpenAI
# from langchain_anthropic import ChatAnthropic
# from langchain_google_genai import ChatGoogleGenerativeAI

### **툴 설정**

In [ ]:
#웹 검색 툴 설정
search_tool = SerperDevTool()

# 사용자 정의 재무분석 툴 로딩
from stock_analyusis import comprehensive_stock_analysis


### **에이전트 설정**

In [ ]:
# 현재 시간 가져오기
current_time = datetime.now()

llm = LLM(model = "openai/gpt-4o-mini")
invest_llm = LLM(model = "openai/o3-mini-2025-01-31")
# invest_llm = LLM(model="anthropic/claude-3-5-sonnet-20240620")

# 재무 분석가
financial_analyst = Agent(
    role="Financial Analyst",
    goal="회사의 재무 상태 및 성과 분석",
    backstory="당신은 재무 제표와 비율을 해석하는 데 전문성을 갖춘 노련한 재무 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# 시장 분석가
market_analyst = Agent(
    role="Market Analyst",
    goal="회사의 시장 지위 및 업계 동향 분석",
    backstory="당신은 기업/산업 현황 및 경쟁 환경을 전문적으로 분석할 수 있는 숙련된 시장 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[search_tool],
    llm=llm,
    max_iter=3,
    allow_delegation=False,
    verbose=True
)

# 위험 분석가
risk_analyst = Agent(
    role="Risk Analyst",
    goal="주식과 관련된 잠재적 위험 식별 및 평가",
    backstory="당신은 투자에서 명백한 위험과 숨겨진 위험을 모두 식별하는 예리한 안목을 갖춘 신중한 위험 분석가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    tools=[comprehensive_stock_analysis],
    llm=llm,
    allow_delegation=False,
    verbose=True
)

# 투자 전문가
investment_advisor = Agent(
    role="Investment Advisor",
    goal="전체 분석을 기반으로 한 투자 추천 제공",
    backstory="다양한 분석을 종합하여 전략적 투자 조언을 제공하는 신뢰할 수 있는 투자 자문가입니다.날짜: {current_time:%Y년 %m월 %d일}",
    llm=invest_llm,
    allow_delegation=False,
    verbose=True
)

In [5]:
def get_user_input():
    ticker = input("투자 자문을 구하고 싶은 기업명을 입력해주세요: ")
    return ticker

def create_dynamic_tasks(ticker):
    financial_analysis = Task(
        description=f"""{ticker}에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=financial_analyst,
        expected_output=f"""{ticker}의 재무 상태에 대한 종합적인 분석 보고서. 
        주요 재무 지표, 수익성, 부채 비율 등을 포함하며, 
        회사의 재무 건전성과 성과 동향에 대한 인사이트를 제공해야 합니다."""
    )

    market_analysis = Task(
        description=f"""{ticker}의 시장 위치를 분석합니다. 
        경쟁 우위, 시장 점유율, 업계 동향을 평가하세요. 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 제공하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=market_analyst,
        expected_output=f"""{ticker}의 시장 위치에 대한 상세한 분석 보고서. 
        경쟁 우위, 시장 점유율, 산업 동향을 평가하고, 
        회사의 성장 잠재력과 시장 과제에 대한 인사이트를 포함해야 합니다."""
    )

    risk_assessment = Task(
        description=f"""{ticker}에 대한 투자와 관련된 주요 위험을 파악하고 평가합니다. 
        시장 위험, 운영 위험, 재무 위험 및 회사별 위험을 고려하세요. 
        종합적인 위험 프로필을 제공합니다.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=risk_analyst,
        expected_output=f"""{ticker} 투자와 관련된 주요 리스크에 대한 포괄적인 평가 보고서. 
        시장 리스크, 운영 리스크, 재무 리스크, 회사 특정 리스크를 고려하여 
        종합적인 리스크 분석 결과를 제시해야 합니다."""
    )

    investment_recommendation = Task(
        description=f"""{ticker}의 재무 분석, 시장 분석, 위험 평가를 바탕으로 종합적인 투자 추천을 제공합니다. 
        주식의 잠재 수익률, 위험 및 다양한 유형의 투자자에 대한 적합성을 고려하세요. 한글로 작성하세요.날짜: {current_time:%Y년 %m월 %d일}""",
        agent=investment_advisor,
        expected_output=f"""
        1. 제목 및 기본 정보
           - 회사명, 티커, 현재 주가, 목표주가, 투자의견 등
        
        2. 요약(Executive Summary)
           - 핵심 투자 포인트와 주요 재무 지표를 간단히 정리
        
        3. 기업 개요
           - 회사의 주요 사업 영역, 연혁, 시장 점유율 등
        
        4. 산업 및 시장 분석
           - 해당 기업이 속한 산업의 트렌드와 전망
        
        5. 재무 분석
           - 매출, 영업이익, 순이익 등 주요 재무지표 분석
           - 수익성, 성장성, 안정성 지표 분석
        
        6. 밸류에이션
           - P/E, P/B, ROE 등 주요 밸류에이션 지표 분석
           - 경쟁사 대비 상대 밸류에이션
        
        7. 투자 의견 및 목표주가
           - 투자의견 제시 및 근거 설명
           - 목표주가 산정 방법과 근거
        
        8. 투자 위험 요인
           - 잠재적인 리스크 요인들을 나열
        
        9. 재무제표 요약
           - 최근 몇 년간의 요약 손익계산서, 재무상태표, 현금흐름표

        """
    )

    return [financial_analysis, market_analysis, risk_assessment, investment_recommendation]

In [ ]:
ticker = get_user_input()
tasks = create_dynamic_tasks(ticker)

crew = Crew(
    agents=[financial_analyst, market_analyst, risk_analyst, investment_advisor],
    tasks=tasks,
    process=Process.sequential,
    output_log_file="logs/investment_advisor_run.json", 
    verbose=False
)

result = crew.kickoff()

# Agent: Financial Analyst
## Task: AAPL에 대한 철저한 재무 분석을 수행합니다. 
        주요 재무 지표에 집중하세요. 
        회사의 재무 건전성 및 성과 추세에 대한 인사이트를 제공합니다.날짜: 2025년 06월 17일


# Agent: Financial Analyst
## Thought: I need to perform a thorough financial analysis of AAPL, focusing on key financial metrics and providing insights into the company’s financial health and performance trends.
## Using tool: Updated Comprehensive Stock Analysis
## Tool Input: 
"{\"ticker\": \"AAPL\"}"
## Tool Output: 
{'현재 주가': {'현재 주가': np.float64(197.10000610351562), '기준 시간': '2025-06-17 00:00:00'}, '연간 데이터': {'매출': '391,035,000,000', '매출원가': '210,352,000,000', '매출총이익': '180,683,000,000', '영업이익': '123,216,000,000', '순이익': '93,736,000,000', 'EBITDA': '134,661,000,000', '매출총이익률': '46.21%', '영업이익률': '31.51%', '순이익률': '23.97%', '매출 성장률': '2.02%', '순이익 성장률': '-3.36%', '희석주당순이익(EPS)': '$6.08', '부채비율': '84.40%'}, '분기 데이터': {'매출': '95,359,000,000', '순이익': '24,780,000,000', '매출 성장률(QoQ)': '-23.28%', '순이익 성장률(QoQ)': '-31.79%'}, '연간 재무제표 요약'

In [7]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

──────────────────────────────
1. 제목 및 기본 정보  
   - 회사명: 애플(Apple Inc.)  
   - 티커: AAPL  
   - 현재 주가: $197.10  
   - 목표주가: $215.00 (보수적 목표 및 성장 잠재력, 기술 혁신 가치를 반영)  
   - 투자의견: 보유 (Hold) – 단기 변동성이 존재하지만, 중장기적으로는 제품 혁신과 효율 개선을 통한 회복 가능성이 있다고 판단

──────────────────────────────
2. 요약 (Executive Summary)  
   애플은 글로벌 테크 기업으로 높은 수익성을 유지하고 있으나, 최근 분기 매출 및 순이익 감소, 그리고 부채 비율이 84.40%에 이르는 등 재무 및 운영 측면에서 일정한 위험을 내포하고 있습니다. 핵심 투자 포인트는 아이폰 및 서비스 부문에서의 브랜드 가치와 기술 혁신이며, 단기 실적 부진으로 인한 주가 조정 기회를 활용할 수 있지만, 중장기 투자자에게는 주의를 기울일 필요가 있습니다.

──────────────────────────────
3. 기업 개요  
   - 애플은 전 세계적으로 잘 알려진 기술 및 소비자 전자제품 분야의 선두 주자입니다.  
   - 주요 사업 영역: 스마트폰(아이폰), 태블릿(아이패드), 컴퓨터(맥), 웨어러블, 서비스(앱스토어, 애플뮤직 등)  
   - 연혁: 혁신적인 제품과 디자인으로 시장을 선도하며 수십 년간 꾸준한 성장을 기록해왔음  
   - 시장 점유율: 스마트폰 및 프리미엄 컴퓨터, 웨어러블 등 주요 분야에서 강력한 브랜드 파워를 보유

──────────────────────────────
4. 산업 및 시장 분석  
   - 애플이 속한 기술 산업은 지속적인 혁신, 빠른 기술 발전, 그리고 경쟁 심화라는 특징을 보임  
   - 최근 글로벌 경기 불황과 공급망 이슈 등으로 단기적으로 매출 및 순이익 감소세를 보이고 있으나, 디지털 전환 및 서비스 부문 확대로 중장기 성장 가능성이 있음  
   - 경쟁사 대비 안정적인 시장 점유율을 유지하고 있으나, 신제품 출시 및 기술 트렌드 변화에 기민하게 대응할 필요가 있음

──────────────────────────────
5. 재무 분석  
   - 주요 수치:  
       · 연간 매출: $391,035,000,000  
       · 영업 이익: $123,216,000,000  
       · 순이익: $93,736,000,000  
       · 매출 총 이익: $180,683,000,000  
       · EBITDA: $134,661,000,000  
   - 성장성: 연간 매출 성장률 2.02%로 안정적이나 연간 순이익은 -3.36%로 감소, 분기별 매출과 순이익 역시 크게 하락함  
   - 수익성: 매출총이익률 46.21%, 영업이익률 31.51%, 순이익률 23.97%로 강력한 수익 창출 능력을 보이나, 단기 실적 하락 우려 존재  
   - 부채 비율: 84.40%로 재무 레버리지 측면에서 주의가 필요하며, 향후 자금 조달 비용 증가 가능성이 있음

──────────────────────────────
6. 밸류에이션  
   - 주요 밸류에이션 지표(P/E, P/B, ROE 등)는 제공된 데이터 외에 추가 조사가 필요하지만, 현재 주가 $197.10 대비 회사의 산업 내 위상과 재무 성과를 고려하면 다소 고평가된 감이 있으나, 브랜드 가치와 지속가능한 현금 흐름은 반영되어 있음  
   - 경쟁사 대비 상대 밸류에이션: 기술 대형주들 중에서도 프리미엄 가격이 매겨지는 기업답게 다소 높은 밸류에이션을 보이나, 장기 투자자에게는 기술 혁신 및 서비스 확장으로 인한 성장 기반이 있는 것으로 평가

──────────────────────────────
7. 투자 의견 및 목표주가  
   - 투자의견: 보유 (Hold)  
       · 단기적으로 시장 변동성과 분기 실적 부진이 우려되나, 중장기적인 관점에서는 기술 혁신 및 서비스 부문의 성장 기회가 충분함  
       · 제품 포트폴리오 다각화와 글로벌 브랜드 신뢰도로 인해 안정적인 현금 흐름 창출이 예상됨  
   - 목표주가 산정:  
       · 보수적 목표 주가 $215.00은 최근 주가 수준에서 약간의 프리미엄을 반영함  
       · 목표 산정 근거는 수익성, 재무 건전성, 업계 내 경쟁력, 그리고 기술 진보에 따른 미래 성장 가능성으로 결정

──────────────────────────────
8. 투자 위험 요인  
   - 시장 위험: 글로벌 경기 둔화, 경쟁 심화, 소비자 수요 변동성  
   - 운영 위험: 공급망 문제 및 글로벌 제조 환경의 불확실성, 높은 부채 비율로 인한 운영 부담  
   - 재무 위험: 영업 이익 및 순이익의 단기 감소세, 부채 증가에 따른 이자 비용 부담  
   - 회사 특정 위험: 제품 혁신 실패, 기술 변화에 따른 시장 점유율 하락, 새로운 경쟁자의 시장 진입 가능성

──────────────────────────────
9. 재무제표 요약  
   - 손익계산서(요약):  
       · 연간 매출: $391,035,000,000  
       · 매출총이익: $180,683,000,000  
       · 영업 이익: $123,216,000,000  
       · 순이익: $93,736,000,000  
       · 분기 실적: 매출 성장률 -23.28%, 순이익 성장률 -31.79%  
   - 재무상태표(요약):  
       · 부채 비율: 84.40% (높은 레버리지 수준으로 운영 안정성에 주의)  
       · 자본과 부채의 균형관계는 안정적이나, 부채 비율 증가는 지속 가능한 성장을 위한 추가적인 전략 모색이 필요  
   - 현금흐름표(요약):  
       · 강력한 영업 현금흐름을 기반으로 하여 투자 및 부채 상환에 활용되고 있으나, 단기 투자 비용과 연구개발비 증가 등으로 인해 현금흐름 변동성이 존재

──────────────────────────────
최종 종합 의견:  
애플은 강력한 브랜드 파워와 우수한 기술 혁신 능력을 바탕으로 한 고수익성을 보유하고 있지만, 최근 단기 실적 하락과 높은 부채 비율은 투자자에게 중요한 경고 신호로 작용합니다. 단기적 불확실성에도 불구하고, 중장기적으로는 기술 리더십 및 서비스 부문 확장을 통해 회복 및 성장할 가능성이 있습니다. 따라서 단기 투자자보다는 중장기적 관점의 투자자에게 더 적합하며, 보유 의견을 유지하되 시장 변화에 따른 지속적인 모니터링이 필요합니다.